In [1]:
import pandas as pd
import numpy as np
import re

In [518]:
with open('../input/test.txt', 'r') as f:
    inp = f.read()

inp[:10]

'##########'

In [519]:
start_pos = inp.split("\n\n")[0].split("\n")
instructions  = ''.join(inp.split("\n\n")[1].split("\n"))

In [520]:
def init_walls(start_pos):
    walls = []
    for x in range(len(start_pos)):
        for y in range(len(start_pos[0])):
            if start_pos[x][y] == '#':
                walls.append((x,y))

    return walls

In [521]:
def init_boxes(start_pos):
    boxes = []
    for x in range(len(start_pos)):
        for y in range(len(start_pos[0])):
            if start_pos[x][y] == 'O':
                boxes.append((x,y))
    return boxes

In [522]:
def init_actor(start_pos):
    for x in range(len(start_pos)):
        for y in range(len(start_pos[0])):
            if start_pos[x][y] == '@':
                return (x,y)

In [523]:
def move(pos, direc):
    if direc == '^':
        return (pos[0]-1, pos[1])
    if direc == '>':
        return (pos[0], pos[1]+1)
    if direc == '<':
        return (pos[0], pos[1]-1)
    if direc == 'v':
        return (pos[0]+1, pos[1])
       

In [524]:
def check_wall(pos):
    if pos in walls:
        return True      
    return False

In [525]:
def check_box(pos):
    if pos in boxes:
        return True
    return False

In [526]:
def move_box(pos, direc):
    next_pos = move(pos, direc)
    if check_wall(next_pos):
        return 0

    if check_box(next_pos):
        move_box(next_pos, direc)

    if not check_box(next_pos):
        boxes.remove(pos)
        boxes.append(next_pos)


In [527]:
def move_actor(pos, direc):
    return move(pos, direc)

In [528]:
walls = init_walls(start_pos)
boxes = init_boxes(start_pos)
actor = init_actor(start_pos)

In [529]:
for direc in instructions:
    next_pos = move(actor, direc)
    if check_wall(next_pos):
        continue
        
    if check_box(next_pos):
        move_box(next_pos, direc)
        
    if not check_box(next_pos):
        actor = move_actor(actor, direc)

In [530]:
def calc_gps_coord(box_coord):
    return 100*box_coord[0] + box_coord[1]

In [531]:
p1 = np.sum([calc_gps_coord(box_coord) for box_coord in boxes])
p1

np.int64(10092)

In [532]:
p2_start_pos = []
for line in start_pos:
    p2_line = line.replace('#', '##').replace('O', '[]').replace('.', '..').replace('@', '@.')
    p2_start_pos.append(p2_line)
p2_start_pos[:10]

['####################',
 '##....[]....[]..[]##',
 '##............[]..##',
 '##..[][]....[]..[]##',
 '##....[]@.....[]..##',
 '##[]##....[]......##',
 '##[]....[]....[]..##',
 '##..[][]..[]..[][]##',
 '##........[]......##',
 '####################']

In [533]:
def init_boxes_p2(start_pos):
    boxes_l = []
    boxes_r = []
    for x in range(len(start_pos)):
        for y in range(len(start_pos[0])):
            if start_pos[x][y] == '[':
                boxes_l.append((x,y))
            if start_pos[x][y] == ']':
                boxes_r.append((x,y))
    return boxes_l, boxes_r

In [534]:
def check_box_p2(pos):
    if pos in boxes_l or pos in boxes_r:
        return True


In [535]:
def move_box_p2(pos, direc):
    next_pos = move(pos, direc)

    if direc == '^' or direc == 'v':
        if check_wall(next_pos):
            return 0

        if pos in boxes_l and check_wall((next_pos[0], next_pos[1]+1)):
            return 0

        if pos in boxes_r and check_wall((next_pos[0], next_pos[1]-1)):
            return 0
        
        if check_box_p2(next_pos):
            move_box_p2(next_pos, direc)

        if pos in boxes_l and check_box_p2((next_pos[0], next_pos[1]+1)):
            move_box_p2((next_pos[0], next_pos[1]+1), direc)

        if pos in boxes_r and check_box_p2((next_pos[0], next_pos[1]-1)):
            move_box_p2((next_pos[0], next_pos[1]-1), direc)

        if (
            not check_box_p2(next_pos) 
            and not (pos in boxes_r and check_box_p2((next_pos[0], next_pos[1]-1))) 
            and not (pos in boxes_l and check_box_p2((next_pos[0], next_pos[1]+1)))
        ):
            if pos in boxes_l:
                boxes_l.remove(pos)
                boxes_l.append(next_pos)

                boxes_r.remove((pos[0], pos[1]+1))
                boxes_r.append((next_pos[0], next_pos[1]+1))
                
            if pos in boxes_r:
                boxes_r.remove(pos)
                boxes_r.append(next_pos)

                boxes_l.remove((pos[0], pos[1]-1))
                boxes_l.append((next_pos[0], next_pos[1]-1))

    if direc == '>' or direc == '<':
        next_pos_2 = move(next_pos, direc)
        if check_wall(next_pos_2):
            return 0

        if check_box_p2(next_pos_2):
            move_box_p2(next_pos_2, direc)

        if not check_box_p2(next_pos_2):
            if pos in boxes_l:
                boxes_l.remove(pos)
                boxes_l.append(next_pos)

                boxes_r.remove(next_pos)
                boxes_r.append(next_pos_2)
                
            if pos in boxes_r:
                boxes_r.remove(pos)
                boxes_r.append(next_pos)

                boxes_l.remove(next_pos)
                boxes_l.append(next_pos_2)

In [547]:
walls = init_walls(p2_start_pos)
boxes_l, boxes_r = init_boxes_p2(p2_start_pos)
actor = init_actor(p2_start_pos)

In [537]:
for direc in instructions:
    next_pos = move(actor, direc)
    if check_wall(next_pos):
        continue
        
    if check_box_p2(next_pos):
        move_box_p2(next_pos, direc)
        
    if not check_box_p2(next_pos):
        actor = move_actor(actor, direc)

In [538]:
p2 = np.sum([calc_gps_coord(box_coord) for box_coord in boxes_l])
p2

np.int64(9021)

In [570]:
list(zip(boxes_l, boxes_r))

[((1, 6), (1, 7)),
 ((1, 12), (1, 13)),
 ((1, 16), (1, 17)),
 ((2, 14), (2, 15)),
 ((3, 12), (3, 13)),
 ((3, 16), (3, 17)),
 ((4, 14), (4, 15)),
 ((5, 2), (5, 3)),
 ((5, 10), (5, 11)),
 ((6, 2), (6, 3)),
 ((6, 8), (6, 9)),
 ((6, 14), (6, 15)),
 ((7, 4), (7, 5)),
 ((7, 6), (7, 7)),
 ((7, 10), (7, 11)),
 ((7, 14), (7, 15)),
 ((7, 16), (7, 17)),
 ((8, 10), (8, 11)),
 ((2, 4), (2, 5)),
 ((4, 5), (4, 6)),
 ((3, 6), (3, 7))]

In [568]:
next_pos = move(actor, direc)
if not check_wall(next_pos):
        
    if check_box_p2(next_pos):
        move_box_p2(next_pos, direc)
        
    if not check_box_p2(next_pos):
        actor = move_actor(actor, direc)

In [569]:
actor

(2, 6)

In [548]:
p2_start_pos

['####################',
 '##....[]....[]..[]##',
 '##............[]..##',
 '##..[][]....[]..[]##',
 '##....[]@.....[]..##',
 '##[]##....[]......##',
 '##[]....[]....[]..##',
 '##..[][]..[]..[][]##',
 '##........[]......##',
 '####################']

In [562]:
direc = 'v'